In [340]:
import pandas as pd
import numpy as np

In [341]:
df=pd.read_csv("D:\Study\Sem 6\Predictive Analytics using Statistics\Sampling_Assignment-main\Creditcard_data.csv")

In [342]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,...,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000
mean,283.005181,-0.176963,0.217169,0.875172,0.285628,-0.005029,0.159081,0.123329,-0.057547,-0.030384,...,0.004888,-0.096995,-0.040344,-0.002501,0.114337,0.022782,0.023353,-0.017045,68.668290,0.011658
std,171.834196,1.294724,1.173401,1.031878,1.258758,1.098143,1.225682,0.852075,0.830144,0.878183,...,0.609335,0.607228,0.358724,0.621507,0.429667,0.484227,0.300934,0.278332,197.838269,0.107411
min,0.000000,-6.093248,-12.114213,-5.694973,-4.657545,-6.631951,-3.498447,-4.925568,-7.494658,-2.770089,...,-4.134608,-2.776923,-3.553381,-1.867208,-1.389079,-1.243924,-2.377933,-2.735623,0.000000,0.000000
25%,126.500000,-0.896416,-0.174684,0.308677,-0.460058,-0.534567,-0.630717,-0.296289,-0.167880,-0.517068,...,-0.213746,-0.525289,-0.176915,-0.379766,-0.166227,-0.313631,-0.047868,-0.033083,5.987500,0.000000
50%,282.000000,-0.382618,0.285843,0.905435,0.395919,-0.116612,-0.109581,0.116329,0.034755,-0.082270,...,-0.075802,-0.076551,-0.048353,0.091886,0.143723,-0.026414,0.023199,0.021034,16.665000,0.000000
75%,432.000000,1.110739,0.885745,1.532969,1.117559,0.452818,0.482972,0.575390,0.252395,0.412261,...,0.095149,0.307438,0.070085,0.426339,0.425798,0.260408,0.112199,0.087023,55.527500,0.000000
max,581.000000,1.586093,5.267376,3.772857,4.075817,7.672544,5.122103,4.808426,2.134599,5.459274,...,5.273420,1.574750,3.150413,1.215279,1.136720,3.087444,2.490503,1.575380,3828.040000,1.000000


Min max scaling of 'Amount' feature

from  sklearn.preprocessing import MinMaxScaler
column = 'Amount'
df[column] = MinMaxScaler().fit_transform(np.array(df[column]).reshape(-1,1))

No. of classes

In [343]:
(df.Class).unique()

array([0, 1], dtype=int64)

## No. of rows pertaining to class=0

In [344]:
len(df[df['Class']==0])

763

## No. of rows pertaining to class=1

In [345]:
len(df[df['Class']==1])

9

class=1 is under-represented (imbalanced dataset)

In [346]:
class_0 = df[df['Class'] == 0]
class_1 = df[df['Class'] == 1]

In [347]:
x=df.drop(['Class'],axis=1)
y=df['Class']

In [348]:
from collections import Counter

## STEP 1 : BALANCING THE DATASET

## Under-sampling methods

# Near-Miss method

Using a distance, this will make the majority class equal to the minority class.

In [349]:
from imblearn.under_sampling import NearMiss

# define the undersampling method
undersample = NearMiss(version=1)
# transform the dataset
x_nm, y_nm = undersample.fit_resample(x,y)
# summarize the new class distribution
print('original dataset shape:', Counter(y))
print('Resample dataset shape', Counter(y_nm))


original dataset shape: Counter({0: 763, 1: 9})
Resample dataset shape Counter({0: 9, 1: 9})


Near-miss under-samples the mojority class(0) to a vary low count,therefore, it is not preferrred

## Tomek-links method
Tomek links are pairs of very close instances but of opposite classes. Removing the instances of the majority class of each pair increases the space between the two classes, facilitating the classification process.

In [350]:
from imblearn.under_sampling import TomekLinks

undersample = TomekLinks()
# transform the dataset
x_tl, y_tl = undersample.fit_resample(x, y)
print('original dataset shape:', Counter(y))
print('Resample dataset shape', Counter(y_tl))

original dataset shape: Counter({0: 763, 1: 9})
Resample dataset shape Counter({0: 756, 1: 9})


## One-sided selection method (OSS)
OSS for short, is an undersampling technique that combines Tomek Links and the Condensed Nearest Neighbor (CNN) Rule.

Tomek Links are ambiguous points on the class boundary and are identified and removed in the majority class. The CNN method is then used to remove redundant examples from the majority class that are far from the decision boundary.

In [351]:
from imblearn.under_sampling import OneSidedSelection

undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200)
x_oss, y_oss = undersample.fit_resample(x, y)
print('original dataset shape:', Counter(y))
print('Resample dataset shape', Counter(y_oss))

original dataset shape: Counter({0: 763, 1: 9})
Resample dataset shape Counter({0: 209, 1: 9})


## Over-sampling methods

## Synthetic Minority Oversampling Technique (SMOTE)
This technique generates synthetic data for the minority class.
It works by randomly picking a point from the minority class and computing the k-nearest neighbors for this point. The synthetic points are added between the chosen point and its neighbors.

(a) SMOTE is applied on the result obtained from Tomek-link under-sampling
This generates a balanced dataset

## Balanced dataset 1

In [352]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy=0.85)

# fit predictor and target variable
x_tl_smote, y_tl_smote = smote.fit_resample(x_tl, y_tl)

print('Original dataset shape', Counter(y_tl))
print('Resample dataset shape', Counter(y_tl_smote))
b_ds1=pd.concat([x_tl_smote, y_tl_smote], axis=1)
print('No. of rows in balanced dataset 1:',len(b_ds1))

Original dataset shape Counter({0: 756, 1: 9})
Resample dataset shape Counter({0: 756, 1: 642})
No. of rows in balanced dataset 1: 1398


(b) SMOTE is applied on the result obtained from OSS under-sampling
This generates a balanced dataset

## Balanced dataset 2

In [353]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy=0.90)

# fit predictor and target variable
x_oss_smote, y_oss_smote = smote.fit_resample(x_oss, y_oss)

print('Original dataset shape', Counter(y_oss))
print('Resample dataset shape', Counter(y_oss_smote))
b_ds2=pd.concat([x_oss_smote, y_oss_smote], axis=1)
print('No. of rows in balanced dataset 2:',len(b_ds2))

Original dataset shape Counter({0: 209, 1: 9})
Resample dataset shape Counter({0: 209, 1: 188})
No. of rows in balanced dataset 2: 397


(c) SMOTE and Random Under Sampling is applied on the original dataset
This generates a balanced dataset

## Balanced dataset 3

In [354]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

over = SMOTE(sampling_strategy=0.50)
under = RandomUnderSampler(sampling_strategy=0.8)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
X, Y = pipeline.fit_resample(x, y)
print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(Y))
b_ds3=pd.concat([X,Y], axis=1)
print('No. of rows in balanced dataset 3:',len(b_ds3))

Original dataset shape Counter({0: 763, 1: 9})
Resample dataset shape Counter({0: 476, 1: 381})
No. of rows in balanced dataset 3: 857


## STEP 2 : CREATING 5 SAMPLES

Sample size (S)=[z^2*p*(1-p)]/E^2
adjusted sample size (AS)=S/[1+((S-1)/POPULATION)]

Z=1.96 (for 95% confidence)
E=0.09 (Margin of error)
p=0.5
S=[(1.96^2)*0.5*0.5]/(0.05^2)
S=384

 For Balanced dataset 1 (Population=1398)
AS=384/[1+(383/1398)]=301

 For Balanced dataset 2 (Population=410)
AS=384/[1+(383/410)]=199

 For Balanced dataset 3 (Population=857)
AS=384/[1+(383/857)]=265

 Sampling done from the 1st balanced dataset

# Simple random sampling

In [355]:
simple_random_sample = b_ds2.sample(n=301)

# Systematic sampling

In [356]:
k=1398/(199)
print('Step size :',int(k))         
def Systematic_sampling(df, step):
    indexes = np.arange(0,len(df),step=step)
    Systematic_sample = df.iloc[indexes]
    return Systematic_sample
Systematic_sample = Systematic_sampling(b_ds1, k)

Step size : 7


# Stratified sampling
No. of strata=2 (class=0, class=1)
sample size= 301*(2^2)

In [357]:
from sklearn.model_selection import StratifiedShuffleSplit
# Set the split criteria
split = StratifiedShuffleSplit(n_splits=1, test_size=301*2*2)
# Perform data frame split
for x, y in split.split(b_ds1, b_ds1['Class']):
    stratified_sample = b_ds1.iloc[y].sort_values(by='Class')

#stratified_sample

print(len(stratified_sample[stratified_sample['Class']==0]))
print(len(stratified_sample[stratified_sample['Class']==1]))

651
553


# Cluster sampling
sample size= 301*c^2

In [358]:
def cluster_sampling(data, cluster_size, num_clusters, sample_size):
    """
    cluster_size : The number of individuals in each cluster.
    num_clusters : The total number of clusters in the population.
    sample_size :  The desired sample size.
    """
    # create clusters
    clusters = np.arange(num_clusters)
    np.random.shuffle(clusters)

    # select sample clusters
    sample_clusters = clusters[:sample_size // cluster_size]

    # select individuals within selected clusters
    sample_individuals = []
    for cluster in sample_clusters:
        start_index = cluster * cluster_size
        end_index = start_index + cluster_size
        individuals = data.iloc[start_index:end_index]
        sample_individuals.append(individuals)

    # concatenate selected individuals into sample dataset
    sample_data = pd.concat(sample_individuals)

    return sample_data

In [359]:
cluster_sample=cluster_sampling(b_ds1,25,55,301*25*25)
len(cluster_sample)

1375

In [360]:
print(len(cluster_sample[cluster_sample['Class']==0]))
print(len(cluster_sample[cluster_sample['Class']==1]))

756
619


## STEP 3 : APPLYING BINARY CLASSIFICATION MODELS

 For SIMPLE RANDOM SAMPLING

In [361]:
from sklearn.model_selection import train_test_split
x1=simple_random_sample.drop(['Class'],axis=1)
y1=simple_random_sample.Class
rs_x_train, rs_x_test, rs_y_train, rs_y_test = train_test_split(x1,y1, random_state=0, test_size=0.2)

Decision Tree

In [362]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

dt = DecisionTreeClassifier(max_depth=2, ccp_alpha=0.01,criterion='gini')
dt.fit(rs_x_train, rs_y_train)
y_pred = dt.predict(rs_x_test)
print(accuracy_score(rs_y_test, y_pred))
print(classification_report(rs_y_test, y_pred))

0.8852459016393442
              precision    recall  f1-score   support

           0       0.93      0.83      0.88        30
           1       0.85      0.94      0.89        31

    accuracy                           0.89        61
   macro avg       0.89      0.88      0.88        61
weighted avg       0.89      0.89      0.88        61



Random Forest

In [363]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 500, max_depth = 4, max_features = 3, bootstrap = True, random_state = 18)
clf.fit(rs_x_train,rs_y_train)
y_pred = dt.predict(rs_x_test)
print(accuracy_score(rs_y_test, y_pred))
print(classification_report(rs_y_test, y_pred))

0.8852459016393442
              precision    recall  f1-score   support

           0       0.93      0.83      0.88        30
           1       0.85      0.94      0.89        31

    accuracy                           0.89        61
   macro avg       0.89      0.88      0.88        61
weighted avg       0.89      0.89      0.88        61



Logistic regression

In [364]:
from sklearn import linear_model
logr = linear_model.LogisticRegression()
logr.fit(rs_x_train,rs_y_train)
predicted = logr.predict(rs_x_test)

print(accuracy_score(rs_y_test, y_pred))
print(classification_report(rs_y_test, predicted))

0.8852459016393442
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.88      0.97      0.92        31

    accuracy                           0.92        61
   macro avg       0.92      0.92      0.92        61
weighted avg       0.92      0.92      0.92        61



c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM

In [365]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(rs_x_train,rs_y_train)
y_pred = clf.predict(rs_x_test)

print(accuracy_score(rs_y_test, y_pred))
print(classification_report(rs_y_test, y_pred))

0.9508196721311475
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        30
           1       0.91      1.00      0.95        31

    accuracy                           0.95        61
   macro avg       0.96      0.95      0.95        61
weighted avg       0.96      0.95      0.95        61



Naive Bayes

In [366]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(rs_x_train, rs_y_train)
y_pred = gnb.predict(rs_x_test)

print(accuracy_score(rs_y_test, y_pred))
print(classification_report(rs_y_test, y_pred))

0.8360655737704918
              precision    recall  f1-score   support

           0       1.00      0.67      0.80        30
           1       0.76      1.00      0.86        31

    accuracy                           0.84        61
   macro avg       0.88      0.83      0.83        61
weighted avg       0.88      0.84      0.83        61



For SYSTEMATIC SAMPLING


In [367]:
from sklearn.model_selection import train_test_split
x1=Systematic_sample.drop(['Class'],axis=1)
y1=Systematic_sample.Class
x_train, x_test,y_train, y_test = train_test_split(x1,y1, random_state=0, test_size=0.2)


Decision Tree

In [368]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

dt = DecisionTreeClassifier(max_depth=2, ccp_alpha=0.01,criterion='gini')
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
print(dt.score(x_test, y_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.85
0.85
              precision    recall  f1-score   support

           0       1.00      0.70      0.82        20
           1       0.77      1.00      0.87        20

    accuracy                           0.85        40
   macro avg       0.88      0.85      0.85        40
weighted avg       0.88      0.85      0.85        40



Random Forest

In [369]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 500, max_depth = 4, max_features = 3, bootstrap = True, random_state = 18)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.95
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        20
           1       0.95      0.95      0.95        20

    accuracy                           0.95        40
   macro avg       0.95      0.95      0.95        40
weighted avg       0.95      0.95      0.95        40



Logistic Regression

In [370]:
from sklearn import linear_model
logr = linear_model.LogisticRegression()
logr.fit(x_train,y_train)
predicted = logr.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, predicted))

0.95
              precision    recall  f1-score   support

           0       1.00      0.80      0.89        20
           1       0.83      1.00      0.91        20

    accuracy                           0.90        40
   macro avg       0.92      0.90      0.90        40
weighted avg       0.92      0.90      0.90        40



c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM

In [371]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.875
              precision    recall  f1-score   support

           0       1.00      0.75      0.86        20
           1       0.80      1.00      0.89        20

    accuracy                           0.88        40
   macro avg       0.90      0.88      0.87        40
weighted avg       0.90      0.88      0.87        40



Naive Bayes

In [372]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.65
              precision    recall  f1-score   support

           0       0.88      0.35      0.50        20
           1       0.59      0.95      0.73        20

    accuracy                           0.65        40
   macro avg       0.73      0.65      0.62        40
weighted avg       0.73      0.65      0.62        40



For STRATIFIED SAMPLING

In [373]:
from sklearn.model_selection import train_test_split
x1=stratified_sample.drop(['Class'],axis=1)
y1=stratified_sample.Class
x_train, x_test,y_train, y_test = train_test_split(x1,y1, random_state=0, test_size=0.2)

Decision Tree

In [374]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

dt = DecisionTreeClassifier(max_depth=2, ccp_alpha=0.01,criterion='gini')
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
print(dt.score(x_test, y_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8755186721991701
0.8755186721991701
              precision    recall  f1-score   support

           0       0.98      0.77      0.86       123
           1       0.81      0.98      0.89       118

    accuracy                           0.88       241
   macro avg       0.89      0.88      0.87       241
weighted avg       0.89      0.88      0.87       241



Random Forest

In [375]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 500, max_depth = 4, max_features = 3, bootstrap = True, random_state = 18)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.975103734439834
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       123
           1       0.96      0.99      0.97       118

    accuracy                           0.98       241
   macro avg       0.98      0.98      0.98       241
weighted avg       0.98      0.98      0.98       241



Logistic Regression

In [376]:
from sklearn import linear_model
logr = linear_model.LogisticRegression()
logr.fit(x_train,y_train)
predicted = logr.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, predicted))

0.975103734439834
              precision    recall  f1-score   support

           0       0.97      0.86      0.91       123
           1       0.87      0.97      0.92       118

    accuracy                           0.92       241
   macro avg       0.92      0.92      0.92       241
weighted avg       0.92      0.92      0.92       241



c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM

In [377]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9294605809128631
              precision    recall  f1-score   support

           0       1.00      0.86      0.93       123
           1       0.87      1.00      0.93       118

    accuracy                           0.93       241
   macro avg       0.94      0.93      0.93       241
weighted avg       0.94      0.93      0.93       241



Naive Bayes

In [378]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8464730290456431
              precision    recall  f1-score   support

           0       0.98      0.72      0.83       123
           1       0.77      0.98      0.86       118

    accuracy                           0.85       241
   macro avg       0.87      0.85      0.84       241
weighted avg       0.88      0.85      0.84       241



For CLUSTER SAMPLING

In [379]:
from sklearn.model_selection import train_test_split
x1=cluster_sample.drop(['Class'],axis=1)
y1=cluster_sample.Class
x_train, x_test,y_train, y_test = train_test_split(x1,y1, random_state=0, test_size=0.2)

Decision Tree

In [380]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

dt = DecisionTreeClassifier(max_depth=2, ccp_alpha=0.01,criterion='gini')
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
print(dt.score(x_test, y_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8981818181818182
0.8981818181818182
              precision    recall  f1-score   support

           0       0.99      0.83      0.90       156
           1       0.81      0.99      0.89       119

    accuracy                           0.90       275
   macro avg       0.90      0.91      0.90       275
weighted avg       0.92      0.90      0.90       275



Random Forest

In [381]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 500, max_depth = 4, max_features = 3, bootstrap = True, random_state = 18)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9745454545454545
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       156
           1       0.95      0.99      0.97       119

    accuracy                           0.97       275
   macro avg       0.97      0.98      0.97       275
weighted avg       0.98      0.97      0.97       275



Logistic Regression

In [382]:
from sklearn import linear_model
logr = linear_model.LogisticRegression()
logr.fit(x_train,y_train)
predicted = logr.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, predicted))

0.9745454545454545
              precision    recall  f1-score   support

           0       0.93      0.88      0.90       156
           1       0.85      0.92      0.88       119

    accuracy                           0.89       275
   macro avg       0.89      0.90      0.89       275
weighted avg       0.90      0.89      0.89       275



c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM

In [383]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9236363636363636
              precision    recall  f1-score   support

           0       1.00      0.87      0.93       156
           1       0.85      1.00      0.92       119

    accuracy                           0.92       275
   macro avg       0.93      0.93      0.92       275
weighted avg       0.94      0.92      0.92       275



Naive Bayes

In [384]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8109090909090909
              precision    recall  f1-score   support

           0       0.96      0.70      0.81       156
           1       0.71      0.96      0.81       119

    accuracy                           0.81       275
   macro avg       0.83      0.83      0.81       275
weighted avg       0.85      0.81      0.81       275

